In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name="quick-starts-ws-165838")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-167587
Azure region: southcentralus
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-167587


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
from azureml.core import ScriptRunConfig
from azureml.core import Environment
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C':uniform(0.1,1),
'--max_iter':quniform(100,1500,100)}

)

# Specify a Policy
policy = early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

script_folder='./training'
os.makedirs(script_folder, exist_ok=True)
import shutil
shutil.copy('./train.py', script_folder)

env=Environment.get(workspace=ws, name="AzureML-Tutorial")

src= ScriptRunConfig(source_directory= script_folder,
                    script='train.py',
                    compute_target= aml_compute,
                     environment=env
                    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy", 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=15,
                             max_concurrent_runs=3)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdr=exp.submit(config=hyperdrive_config)

In [5]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.
best_run=hdr.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments']) 
print(best_run.get_file_names())

model=best_run.register_model(model_name="Best", model_path=".")

best_run_metrics= best_run.get_metrics()
print('\n Accuracy: ', best_run_metrics['Accuracy'])

### YOUR CODE HERE ###

['--C', '0.3338893693170834', '--max_iter', '1000']
['azureml-logs/55_azureml-execution-tvmps_faccb655a07fbfdd0eb0062fd9a7aeb02a39441957eddc2c6a049fe4ceaffc97_d.txt', 'azureml-logs/65_job_prep-tvmps_faccb655a07fbfdd0eb0062fd9a7aeb02a39441957eddc2c6a049fe4ceaffc97_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_faccb655a07fbfdd0eb0062fd9a7aeb02a39441957eddc2c6a049fe4ceaffc97_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/114_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']

 Accuracy:  0.9098634294385433


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds=TabularDatasetFactory.from_delimited_files(["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])

In [9]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

#Split into train and test datasets
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.20)

train_df=pd.concat([x_train, y_train], axis=1)
test_df=pd.concat([x_test, y_test], axis=1)

train_df.head()




,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
16515,56,1,0,0,0,5,3,167,3,999,...,1,0,0,1,0,0,0,0,0,0
26617,32,1,0,0,0,8,4,172,3,999,...,0,0,0,0,0,0,1,0,0,0
3167,31,1,0,1,0,5,5,191,1,999,...,0,0,1,0,0,0,0,0,0,0
21001,36,1,0,0,0,6,4,22,1,999,...,1,0,0,0,0,0,0,1,0,0
1695,47,1,0,0,0,6,1,200,1,999,...,1,1,0,0,0,0,0,0,0,0


In [10]:
#save the dataframe as csv and upload to datastore
if not os.path.isdir('training'):
    os.mkdir('training')
pd.DataFrame(train_df).to_csv("training/train_df.csv", index=False)
pd.DataFrame(test_df).to_csv("training/test_df.csv", index=False)

data_store=ws.get_default_datastore()
data_store.upload(src_dir='./training', target_path='bankmarketing', overwrite=True, show_progress=True)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 3 files
Uploading ./training/test_df.csv
Uploaded ./training/test_df.csv, 1 files out of an estimated total of 3
Uploading ./training/train.py
Uploaded ./training/train.py, 2 files out of an estimated total of 3
Uploading ./training/train_df.csv
Uploaded ./training/train_df.csv, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_e4dd3dd157da471291e794eb39906853

In [11]:
from azureml.core.dataset import Dataset

train_df=Dataset.Tabular.from_delimited_files(path=data_store.path('bankmarketing/train_df.csv'))

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_df,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=aml_compute 
    )

In [13]:
# Submit your automl run

### YOUR CODE HERE ###
auto_exp=Experiment(workspace=ws, name="auto_exp")
automl_run=exp.submit(automl_config, show_output=False)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-165838,AutoML_b48bb646-43bf-42a6-abde-7bd9660356e1,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_b48bb646-43bf-42a6-abde-7bd9660356e1',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-12-21T20:36:44.886031Z',
 'endTimeUtc': '2021-12-21T21:17:00.230597Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"quick-starts-ws-165838","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group":"aml-quickstarts-167587","workspace_name":"quick-starts-ws-167587","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_si

In [14]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_model, fitted_model=automl_run.get_output()
print(best_model)
print(fitted_model)
best_run_metrics=best_run.get_metrics()

best_model.register_model(model_name="best_auto_model", model_path="./outputs")

Run(Experiment: quick-starts-ws-165838,
Id: AutoML_b48bb646-43bf-42a6-abde-7bd9660356e1_22,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))], verbose=False)), ('19', Pipeline(memory=None, steps=[('truncatedsvdwrapper', TruncatedSVDWrapper(n_components=0.8015789473684211, random_state=None)), ('randomforestclassifier', RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced', criterion='entropy', max_depth=None, max_features='sqrt', max_leaf_nodes=None, max_samples=None, min_impurity_de

Model(workspace=Workspace.create(name='quick-starts-ws-167587', subscription_id='61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30', resource_group='aml-quickstarts-167587'), name=best_auto_model, id=best_auto_model:1, version=1, tags={}, properties={})